## Trabalhando com Big Data - 25 Milhões de Registros
fonte: https://grouplens.org/datasets/movielens/

In [ ]:
import pandas as pd

In [ ]:
!pwd

In [ ]:
!ls -ilah /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m

In [ ]:
!head /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/movies.csv

In [ ]:
url = '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m' + '/' + 'ratings.csv'
url

In [ ]:
%time

dfratings = pd.read_csv(url)
print('total de registros: {:,d}'.format(len(dfratings)))

In [ ]:
dfratings.head()

In [ ]:
url = '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m' + '/' + 'movies.csv'
url

In [ ]:
%time

dfmovies = pd.read_csv(url)
print('total de registros: {:,d}'.format(len(dfmovies)))

In [ ]:
dfmovies.head()

In [ ]:
%%time

dfdrama_acao = dfmovies[ dfmovies.genres.str.contains('Drama') |  dfmovies.genres.str.contains('Action') ]
print('total de registros: {:,d}'.format(len(dfdrama_acao)))
dfdrama_acao.head()

# Obter todos os generos

In [ ]:
genres = 'Comedy|Drama|Romance'
generos = genres.split('|')
generos

In [ ]:
%%time

set_generos = set()

for index, row in dfmovies.iterrows(): 
    generos = row['genres'].split('|')
    set_generos.update(generos)
    
generos = list(set_generos)
print('numero de generos',len(generos))
print(generos)

## Criando colunas Genero - individuais

In [ ]:
%%time

for genero in generos:
    dfmovies[genero] = False
    
dfmovies.head()

In [ ]:
%%time 

for index, row in dfmovies.iterrows(): 
    generos = row['genres'].split('|')
    for genero in generos:
        dfmovies.at[index, genero] = True
    
dfmovies.head()

In [ ]:
dfmovies.Drama.value_counts()

In [ ]:
del dfmovies['genres']
dfmovies.head()

# criar estatistica de generos

In [ ]:
generos = list(set_generos)
print('numero de generos',len(generos))
print(generos)

In [ ]:
dfmovies.Horror.value_counts()

In [ ]:
for genero in generos:
    count_genero = dfmovies[genero].sum()
    print(genero, count_genero)
    break

In [ ]:
dict_genero = {}
for genero in generos:
    count_genero = dfmovies[genero].sum()
    dict_genero[genero] = count_genero

dict_genero

In [ ]:
df_genero_cont = pd.DataFrame.from_dict(dict_genero, orient='index').reset_index()
df_genero_cont

In [ ]:
df_genero_cont.columns = ['genero', 'total']
df_genero_cont

In [ ]:
dfo = df_genero_cont.sort_values(by='total', ascending=False).reset_index(drop=True)
dfo

# Capturando o ano

In [ ]:
dfmovies.head()

In [ ]:
x = 'Father of the Bride Part II (1995)'
x.find(')')

In [ ]:
type(x)

In [ ]:
x = 'Father of the Bride Part II'
x.find(')')

In [ ]:
import numpy as np

dfmovies['ano'] = dfmovies.title.apply(lambda x : x.split('(')[1] if x.find(')') != -1  else np.nan)
dfmovies.head()

In [ ]:
dfmovies['ano'] = dfmovies.ano.apply(lambda x : x.replace(')', '') if type(x) == str else x)
dfmovies.head()

In [ ]:
print(dfmovies.ano.value_counts())

In [ ]:
import re

x1 = '19993'
x2 = '1 2 title'
datas = r'^([1-9][0-9][0-9][0-9])'
#re.match(r'\d{4}', x)

if re.match(datas, x1):
    print('ok')
if re.match(datas, x2):
    print('ok nao data')

In [ ]:
datas = r'^([0-9][0-9][0-9][0-9])'

dfmovies['ano'] = dfmovies.title.apply(lambda x : x.split('(')[1] if x.find(')') != -1  else np.nan)
dfmovies['ano'] = dfmovies.ano.apply(lambda x : x.replace(')', '') if type(x) == str else x)

dfmovies['ano'] = dfmovies.ano.apply(lambda x : x[:4] if x is not np.nan and re.match(datas, x[:4]) else np.nan)
dfmovies.head()

In [ ]:
dfmovies.ano.value_counts()

In [ ]:
len(dfmovies.ano.unique())

In [ ]:
print(dfmovies.ano.unique())

In [ ]:
dfmovies.ano.isnull().sum(), len(dfmovies.ano)

In [ ]:
dfanofilmes = dfmovies.ano.value_counts().to_frame().reset_index()
dfanofilmes.columns = ['ano', 'total_filmes']
dfanofilmes.head()

In [ ]:
dfanofilmes = dfanofilmes.sort_values(by='total_filmes', ascending=False).reset_index(drop=True)
dfanofilmes.head(10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,8))

sns.barplot(x='total_filmes', y='ano', data=dfanofilmes[:20], palette='Greens_r')

# Join - inner join

In [ ]:
dfratings.head()

In [ ]:
dfmovies.head()

In [ ]:
%%time

dffilmes =  pd.merge(dfmovies, dfratings, left_on='movieId', right_on='movieId', how="inner")
print('total de registros: {:,d}'.format(len(dffilmes)))
dffilmes.head()

In [ ]:
dffilmes.info()

In [ ]:
dffilmes.sort_values(by='rating', ascending=False)[:15]

In [ ]:
# generos = ['Crime', 'Western', 'Thriller', 'War', 'Sci-Fi', 'Children', 
#             'Documentary', '(no genres listed)', 'Romance', 'Adventure', 'Mystery',
#             'Comedy', 'IMAX', 'Animation', 'Action', 'Horror', 'Film-Noir', 'Fantasy',
#         'Drama', 'Musical'] 

In [ ]:
%%time

dffilmes.to_csv('filmes-2022.csv', index=False)
!ls -ilah *.csv

In [ ]:
dffilmes.Action.value_counts()

In [ ]:
acao = dffilmes.Action.value_counts()[1]
acao

In [ ]:
dffilmes.Drama.value_counts()

In [ ]:
drama = dffilmes.Drama.value_counts()[1]
drama

In [ ]:
dffilmes.Adventure.value_counts()

In [ ]:
aventura = dffilmes.Adventure.value_counts()[1]
aventura

In [ ]:
dffilmes.Romance.value_counts()

In [ ]:
romance = dffilmes.Romance.value_counts()[1]
romance

In [ ]:
#outros = len(dfmovies) - aventura - drama - romance - acao
#outros

# grafico de Pizza - genero

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

plt.rcParams["figure.figsize"] = (9,9)

#define data
data =    [aventura, drama, romance, acao] #f6.percent
labels = ['aventura', 'drama', 'romance', 'acao'] #, 'outros'] #df.rating

#define Seaborn color palette to use
colors = sns.color_palette('pastel')#[0:4]

colors = sns.color_palette("rocket")

colors = sns.color_palette("Blues_r") #, as_cmap=True)

#colors = 'Blues'



fig = plt.figure()

#ax = fig.add_axes([.8, .8, .8, .8], aspect=2)

ax = fig.add_axes([.8, .8, .8, .8], aspect=2)
ax.set_title('Filmes por genero ')
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')


plt.show()

In [ ]:
dfo = dffilmes.rating.value_counts()
dfo

In [ ]:
type(dfo)

In [ ]:
df2 = dfo.to_frame().reset_index()
df2.columns = ['rating', 'quantidade']

In [ ]:
df2

In [ ]:
df2['percent'] = df2['quantidade'] / df2['quantidade'].sum() * 100
df2

In [ ]:
# formatar a saída

df3 = df2.style.format({
    "rating": "{:.1f}",
    "quantidade": "{:,d}",
    "percent": "{:.0f}"
})



df3

In [ ]:
df4 = df2[ df2.percent >= 7]
df4

In [ ]:
df4.percent.sum()

In [ ]:
print('{:,d}'.format(df4.quantidade.sum()))

In [ ]:
df5 = df2[ df2.percent < 7]
df5

In [ ]:
print('{:,d}'.format(df5.quantidade.sum()))

In [ ]:
df5.percent.sum()

In [ ]:
df5

In [ ]:
row = {'rating': 'outros', 'quantidade': df5.quantidade.sum(), 'percent': df5.percent.sum()  }
row

In [ ]:
df6 = df4.append(row, ignore_index=True)
df6

In [ ]:
# formatar a saída

dfs = df6.style.format({
    "rating": "{}",
    "quantidade": "{:,d}",
    "percent": "{:.0f}"
})

dfs

In [ ]:
df6.to_csv('ratings_saida.csv', index=False)

In [ ]:
!cat ratings_saida.csv

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (9,9)

#define data
data = df6.percent
labels = df6.rating

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:7]

colors = sns.color_palette("Blues_r")

colors = sns.color_palette("Reds_r")[0:7]

colors = sns.color_palette("Blues_r")[0:7]


colors = sns.color_palette("Oranges_r")[0:8]


fig = plt.figure()

ax = fig.add_axes([0, 0, .8, .8], aspect=1)

ax.set_title('Filmes por avaliação (1-5)')
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')


plt.show()

# 1 Bilhão de Registros

In [ ]:
!ls ~/Downloads/ml-20mx16x32

In [ ]:
pwd

In [ ]:
#!pip install -q npzviewer

In [ ]:
%%time
import numpy as np

url  = '/Users/valencar/Downloads/ml-20mx16x32/'
n = np.load(url + 'testx16x32_0.npz')

In [ ]:
 n.files

In [ ]:
n['arr_0'].shape

In [ ]:
n['arr_0'][20]

In [ ]:
%%time
import pandas as pd

url = 

df = pd.read_csv(url)
df.head